# RL Lecture 3
## Planning by Dynamic Programming
https://www.youtube.com/watch?v=Nd1-UUMVfz4&list=PLqYmG7hTraZDM-OYHWgPebj2MfCFzFObQ&index=3
(David Silver)

Dynamic Programming is a general approach to create algorithms if the problem has a structure in which we can divide it into smaller but similar problems. For example, shortest path could be viewed this way, because in order to find a shortest path from A to B in a graph, it is enough to find a shortest path to from each neighbor of A (or to each neighbor of B) and choose te best one. In this case, it might not be the most intuitive approach to solving this problem, but many problems can be solved using this principle (for example sequence alignment in bioinformatics (Needleman–Wunsch) algorithm, edit ( Levenshtein) distance, longest common subsequence in two strings, Viterbi algorithm and - most importantly for us - solving Bellman equations). More algorithms can be found here: https://en.wikipedia.org/wiki/Dynamic_programming. Even though these problems sound not related at all, seem complicated and have weird names and many authors, they all share the same underlying principle. And this principle was invented by Richard Bellman in 1950s.

Now, about the problem we are trying to solve. We are considering planning and control, given that we have full knowledge of the world (modelled as Markov Decision Process). 

#### Planning
Evaluating policy $\pi$ in the MDP world.

Input: MDP: $(\mathcal{S}, \mathcal{A}, \mathcal{P}, \mathcal{R}, \gamma)$, policy $\pi$

Output: value funciton $v_{\pi}$


#### Control
Actually finding the best policy.

Input: MDP: $(\mathcal{S}, \mathcal{A}, \mathcal{P}, \mathcal{R}, \gamma)$

Output: *optimal* value funciton $v_{\pi}$

Note, that when we have optimal value function, we immediately have the optimal policy: do the action that maximises the value function.

## Example
### Defining MDP Process
Consider a 4x4 grid world. If we are at state 0 or 15 we get a reward of 0 and stay there forever. Otherwise, the reward is -1. In each state we can go up, down, left or right. If it would take us outside of a grid, we stay in the same place.

In [550]:
import numpy as np
import random
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod
from tqdm.notebook import tqdm

grid_world = np.array(np.arange(16)).reshape(4, 4)
grid_world

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])

In [532]:
# note: inheriting from int is completely useless 
# here, just for fun
class State(int):
    def __new__(cls, row, col):
        return int.__new__(cls, row * 4 + col)
    
    def reward(self):  # in general reward might depend on action (but not in this example)
        if self == 0 or self == 15:
            return 0
        else:
            return -1
        
    @property
    def position(self):
        return divmod(self, 4)

class Action(ABC):
    @abstractmethod
    def direction(self):
        pass

class N(Action):
    def direction(self):
        return -1, 0
    
    def __repr__(self):
        return "N"
    pass

class S(Action):
    def direction(self):
        return 1, 0
    
    def __repr__(self):
        return "S"
    pass

class E(Action):
    def direction(self):
        return 0, 1
    
    def __repr__(self):
        return "E"
    pass

class W(Action):
    def direction(self):
        return 0, -1
    
    def __repr__(self):
        return "W"
    pass

actions = [N(), S(), W(), E()]

grid_world = np.array([[State(*divmod(s, 4)) for s in row] for row in grid_world])

In [533]:
def is_valid_position(row, col):
    return 0 <= row < 4 and 0 <= col < 4

def take_action(current_state: State, action: Action) -> State:
    row, col = current_state.position
    if (row, col) == (0, 0) or (row, col) == (3, 3):
        return current_state
    dirs = action.direction()
    new_position = row + dirs[0], col + dirs[1]
    if is_valid_position(*new_position):
        return State(row + dirs[0], col + dirs[1])
    else:
        return current_state

### Stochastic policy
In each step we pick action randomly with equal probabilities.

In [534]:
policy_vector = np.ones(4) / 4

def policy(state: State) -> Action:
    return np.random.choice(actions, p=policy_vector)

### Value function
We initalize the value function with all zeros.

In [535]:
# Initializing value function
values = np.zeros_like(grid_world, dtype=np.float32)
values

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)

### Bellman equation for value function
In general, if state after action was stochastic, we would have to average over all possible outcomes of an action.
$$ v^{k+1}(s) = \sum_{a \in \mathcal{A}} \pi(a|s) (R^a_s + \gamma \sum_{s' \in \mathcal{S}} P^a_{ss'} v_k(s')) $$

In the case when the state after taking action is deterministic however, we obtain have only one element in the inner sum. It all comes down to averagin over neighboring states.


In [536]:
gamma = 1  # no discount factor
old_values = values.copy()
values = np.zeros_like(values)
# Iterate over states
for row in range(0, 4):
    for col in range(0, 4):
        for action, prob in zip(actions, policy_vector):
            s = State(row, col)
            new_state = take_action(s, action)
            new_row, new_col = new_state.position
            old_value = old_values[new_row][new_col]
            values[row, col] += prob * (s.reward() + gamma * old_value)  # in general, we would have to average over possible outcomes of our action

In [537]:
values

array([[ 0., -1., -1., -1.],
       [-1., -1., -1., -1.],
       [-1., -1., -1., -1.],
       [-1., -1., -1.,  0.]], dtype=float32)

Above, we performed one iteration of value update according to Bellman equation. We see, that the value of the first and last state stayed $0$, but the rest turned to $1$, that is immediate reward of being in this place. Let's perform more iterations and see how the value of this random policy changes when we do more iterations.

In [538]:
def update_values(values):
    old_values = values.copy()
    values = np.zeros_like(values)
    # Iterate over states
    for row in range(0, 4):
        for col in range(0, 4):
            for action, prob in zip(actions, policy_vector):
                s = State(row, col)
                new_state = take_action(s, action)
                new_row, new_col = new_state.position
                old_value = old_values[new_row][new_col]
                values[row, col] += prob * (s.reward() + gamma * old_value)  # in general, we would have to average over possible outcomes of our action
    err = np.max((values - old_values) **2)**0.5  # we control maximum value of element-wise error
    return values, err

In [539]:
print("Optimal value function in time until convergence")
values = np.zeros_like(grid_world, dtype=np.float32)
n_iter = 100
for i in tqdm(range(n_iter)):
    values, err = update_values(values)
    if i < 3 or i % 10 == 0:
        print(i)
        print(np.round(values, 1))
        print()
    if err < 0.01:
        break

Optimal value function in time until convergence


0
[[ 0. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1.  0.]]

1
[[ 0.  -1.8 -2.  -2. ]
 [-1.8 -2.  -2.  -2. ]
 [-2.  -2.  -2.  -1.8]
 [-2.  -2.  -1.8  0. ]]

2
[[ 0.  -2.4 -2.9 -3. ]
 [-2.4 -2.9 -3.  -2.9]
 [-2.9 -3.  -2.9 -2.4]
 [-3.  -2.9 -2.4  0. ]]

10
[[ 0.  -6.6 -9.  -9.7]
 [-6.6 -8.3 -9.  -9. ]
 [-9.  -9.  -8.3 -6.6]
 [-9.7 -9.  -6.6  0. ]]

20
[[  0.   -9.7 -13.6 -14.9]
 [ -9.7 -12.4 -13.7 -13.6]
 [-13.6 -13.7 -12.4  -9.7]
 [-14.9 -13.6  -9.7   0. ]]

30
[[  0.  -11.5 -16.3 -17.9]
 [-11.5 -14.7 -16.3 -16.3]
 [-16.3 -16.3 -14.7 -11.5]
 [-17.9 -16.3 -11.5   0. ]]

40
[[  0.  -12.6 -17.9 -19.6]
 [-12.6 -16.1 -17.9 -17.9]
 [-17.9 -17.9 -16.1 -12.6]
 [-19.6 -17.9 -12.6   0. ]]

50
[[  0.  -13.2 -18.8 -20.6]
 [-13.2 -16.9 -18.8 -18.8]
 [-18.8 -18.8 -16.9 -13.2]
 [-20.6 -18.8 -13.2   0. ]]

60
[[  0.  -13.5 -19.3 -21.2]
 [-13.5 -17.4 -19.3 -19.3]
 [-19.3 -19.3 -17.4 -13.5]
 [-21.2 -19.3 -13.5   0. ]]

70
[[  0.  -13.7 -19.6 -21.5]
 [-13.7 -17.6 -19.6 -19.6]
 [-19.6 -

In [540]:
values

array([[  0.      , -13.895283, -19.84483 , -21.826353],
       [-13.895283, -17.863304, -19.845867, -19.84483 ],
       [-19.84483 , -19.845867, -17.863304, -13.895284],
       [-21.826355, -19.84483 , -13.895285,   0.      ]], dtype=float32)

Based on new value function we can improve our policy from random to deterministic

In [548]:
def improved_policy(state_number: int) -> Action:
    # q_function is the value v given the action a
    q_function = []
    for action in actions:
        state = State(*divmod(state_number, 4))
        new_state = take_action(state, action)
        row, col = new_state.position
        v = values[row, col]
        q_function.append((action, v))
    a, v = max(q_function, key=lambda t: t[1])  # action with the highest value
    return a

In [549]:
new_policy = np.vectorize(lambda s: improved_policy(s))(grid_world)
new_policy

array([[N, W, W, S],
       [N, N, S, S],
       [N, N, E, S],
       [N, E, E, N]], dtype=object)

This is the policy that intuitively makes sense, it allows to quickly reach the top-left or bottom-right corners, where the value is the best. Note, that we would obtian this policy even before the convergence of value function (after only three iterations we would be able to obtain this policy).

## General case
In general, we wouldn't be able to obtain optimal policy after computing value function of random policy. However, we would be able to improve that policy, and use it to improve value function once again. This gives us algorithm for finding the best policy.

Here is algorithm and a visualization from David Silver's slide:

![](img/policy_iteration.png)

(https://www.davidsilver.uk/teaching/)